In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Read in all datasets

In [2]:
# All Billboard Top 100 songs from 1958-2024 w/o the necessary song features *source of truth*
data_hits = pd.read_csv('charts_billboard_1958_2024.csv')

# Billboard hits of unspecified data range w/ all features, missing year
df_hits = pd.read_csv('hot_100_audio_features.csv')

# Dataset of ~1.2 million songs from 2000-2023 w/ all features
data_1m = pd.read_csv('spotify_data.csv')

# Dataset of 1.2 mil tracks w/ all features
df1 = pd.read_csv('spotify_1million.csv')

### Preliminary clean-up of datasets (column formatting, filtering, etc)

#### cleaning up messy billboard dataset that contains audio features

In [3]:
df_hits.columns

Index(['SongID', 'Performer', 'Song', 'spotify_genre', 'spotify_track_id',
       'spotify_track_preview_url', 'duration_ms', 'explicit', 'album',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'spotify_track_popularity'],
      dtype='object')

In [4]:
df_hits = df_hits[['Performer', 'Song', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]

In [5]:
df_hits = df_hits.dropna()

In [6]:
len(df_hits)

24330

#### cleaning artists column in million track dataset to remove random characters

In [7]:
df1.describe()

,track_number,disc_number,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year
count,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06,1.204025e+06
mean,7.656352e+00,1.055906e+00,4.930565e-01,5.095363e-01,5.194151e+00,-1.180870e+01,6.714595e-01,8.438219e-02,4.467511e-01,2.828605e-01,2.015994e-01,4.279866e-01,1.176344e+02,2.488399e+05,3.832494e+00,2.007328e+03
std,5.994977e+00,2.953752e-01,1.896694e-01,2.946839e-01,3.536731e+00,6.982132e+00,4.696827e-01,1.159914e-01,3.852014e-01,3.762844e-01,1.804591e-01,2.704846e-01,3.093705e+01,1.622104e+05,5.611826e-01,1.210117e+01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+03,0.000000e+00,0.000000e+00
25%,3.000000e+00,1.000000e+00,3.560000e-01,2.520000e-01,2.000000e+00,-1.525400e+01,0.000000e+00,3.510000e-02,3.760000e-02,7.600000e-06,9.680000e-02,1.910000e-01,9.405400e+01,1.740900e+05,4.000000e+00,2.002000e+03
50%,7.000000e+00,1.000000e+00,5.010000e-01,5.240000e-01,5.000000e+00,-9.791000e+00,1.000000e+00,4.460000e-02,3.890000e-01,8.080000e-03,1.250000e-01,4.030000e-01,1.167260e+02,2.243390e+05,4.000000e+00,2.009000e+03
75%,1.000000e+01,1.000000e+00,6.330000e-01,7.660000e-01,8.000000e+00,-6.717000e+00,1.000000e+00,7.230000e-02,8.610000e-01,7.190000e-01,2.450000e-01,6.440000e-01,1.370460e+02,2.858400e+05,4.000000e+00,2.015000e+03
max,5.000000e+01,1.300000e+01,1.000000e+00,1.000000e+00,1.100000e+01,7.234000e+00,1.000000e+00,9.690000e-01,9.960000e-01,1.000000e+00,1.000000e+00,1.000000e+00,2.489340e+02,6.061090e+06,5.000000e+00,2.020000e+03


In [8]:
# finding tracks where year is returning zero
df1[df1['year'] == 0]

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
815351,035h5flqzwF6I5CTfsdHPA,Jimmy Neutron,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],1,1,False,0.795,...,0.0519,0.01560,0.439,0.0860,0.389,109.985,183000,4.0,0,0000
815352,49x05fLGDKCsCUA7CG0VpY,I Luv You,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],2,1,False,0.762,...,0.0950,0.88700,0.909,0.1060,0.728,92.962,145161,4.0,0,0000
815353,4mNLlSoZOqoPauBAF3bIpx,My Heart,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],3,1,False,0.671,...,0.0662,0.00956,0.902,0.0455,0.893,97.865,176561,4.0,0,0000
815354,7w5iwI0wnIiopbCFNe1Txo,I Am (Invincible),Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],4,1,False,0.759,...,0.1280,0.00544,0.895,0.0538,0.537,89.989,192000,4.0,0,0000
815355,2Tfy2R2uiWVwxHQUT6oGNp,Flower Power,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],5,1,False,0.657,...,0.2810,0.01800,0.245,0.2410,0.964,179.904,138666,4.0,0,0000
815356,05cTbSPQyha6z7opYwH67O,Heard It Low,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],6,1,False,0.728,...,0.0673,0.00785,0.275,0.0865,0.662,90.010,138667,4.0,0,0000
815357,1fYK5xB8csOXVEqApkzzm0,Hangin On,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],7,1,False,0.822,...,0.0758,0.11500,0.881,0.1210,0.766,119.998,142620,4.0,0,0000
815358,4G51c7cWzB6CLaRq9sYj2w,God Loves You,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],8,1,False,0.845,...,0.0662,0.00274,0.548,0.0393,0.472,120.090,161000,4.0,0,0000
815359,45fcUAjXlzDxTwSzoUaO6l,You In My Life,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],9,1,False,0.957,...,0.0623,0.13300,0.857,0.0968,0.258,112.987,214867,4.0,0,0000
815360,35TcKSN5hsGcZLrFPkUvIv,I Wonder,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],10,1,False,0.659,...,0.0581,0.00196,0.854,0.3710,0.877,146.020,180822,4.0,0,0000


In [9]:
# only one album is returning year = 0, confirmed from Spotify that Optimism 2 by Icizzle was released 2018 so adding that back in
df1.loc[(df1['year'] == 0) & (df1['album'] == 'Optimism 2') & (df1['artists'].str.contains('iCizzle')), 'year'] = 2018

In [10]:
df1[df1['album'] == 'Optimism 2']

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
815351,035h5flqzwF6I5CTfsdHPA,Jimmy Neutron,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],1,1,False,0.795,...,0.0519,0.01560,0.439,0.0860,0.389,109.985,183000,4.0,2018,0000
815352,49x05fLGDKCsCUA7CG0VpY,I Luv You,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],2,1,False,0.762,...,0.0950,0.88700,0.909,0.1060,0.728,92.962,145161,4.0,2018,0000
815353,4mNLlSoZOqoPauBAF3bIpx,My Heart,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],3,1,False,0.671,...,0.0662,0.00956,0.902,0.0455,0.893,97.865,176561,4.0,2018,0000
815354,7w5iwI0wnIiopbCFNe1Txo,I Am (Invincible),Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],4,1,False,0.759,...,0.1280,0.00544,0.895,0.0538,0.537,89.989,192000,4.0,2018,0000
815355,2Tfy2R2uiWVwxHQUT6oGNp,Flower Power,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],5,1,False,0.657,...,0.2810,0.01800,0.245,0.2410,0.964,179.904,138666,4.0,2018,0000
815356,05cTbSPQyha6z7opYwH67O,Heard It Low,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],6,1,False,0.728,...,0.0673,0.00785,0.275,0.0865,0.662,90.010,138667,4.0,2018,0000
815357,1fYK5xB8csOXVEqApkzzm0,Hangin On,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],7,1,False,0.822,...,0.0758,0.11500,0.881,0.1210,0.766,119.998,142620,4.0,2018,0000
815358,4G51c7cWzB6CLaRq9sYj2w,God Loves You,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],8,1,False,0.845,...,0.0662,0.00274,0.548,0.0393,0.472,120.090,161000,4.0,2018,0000
815359,45fcUAjXlzDxTwSzoUaO6l,You In My Life,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],9,1,False,0.957,...,0.0623,0.13300,0.857,0.0968,0.258,112.987,214867,4.0,2018,0000
815360,35TcKSN5hsGcZLrFPkUvIv,I Wonder,Optimism 2,211vSdhxt58A943r9QWRKo,['iCizzle'],['7arv4matK2uKJrdtPSxU4i'],10,1,False,0.659,...,0.0581,0.00196,0.854,0.3710,0.877,146.020,180822,4.0,2018,0000


In [11]:
# artists column has odd characters, removing those
df1['artists'] = df1['artists'].str.replace(r"[\[\]()']", '', regex=True)
df1.head(2)

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,Rage Against The Machine,['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.0727,0.0261,0.000011,0.356,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,Rage Against The Machine,['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.1880,0.0129,0.000071,0.155,0.489,103.680,206200,4.0,1999,1999-11-02


#### cleaning complete Billboard Top 100 dataset (source of truth) to format year and limit to relevant columns

In [12]:
data_hits.head()

,Date,Song,Artist,Rank,Last Week,Peak Position,Weeks in Charts,Image URL
0,8/6/1958,Poor Little Fool,Ricky Nelson,1,1,1,2,#
1,8/6/1958,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,2,54,2,2,https://charts-static.billboard.com/img/1958/0...
2,8/6/1958,Patricia,Perez Prado And His Orchestra,3,2,2,2,#
3,8/6/1958,Splish Splash,Bobby Darin,4,3,3,2,https://charts-static.billboard.com/img/1958/0...
4,8/6/1958,When,Kalin Twins,5,5,5,2,#


In [13]:
# Add column for year instead of fulll release data to align with other datasets
data_hits['year'] = pd.to_datetime(data_hits['Date'], errors='coerce', format='%m/%d/%Y').dt.year

data_hits.head(1)

,Date,Song,Artist,Rank,Last Week,Peak Position,Weeks in Charts,Image URL,year
0,8/6/1958,Poor Little Fool,Ricky Nelson,1,1,1,2,#,1958


In [14]:
data_hits = data_hits[['Song', 'Artist', 'year']]

#### checking other 1 million song dataset, no upfront manipulation necessary

In [15]:
data_1m.describe()

,Unnamed: 0,popularity,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06,1.159764e+06
mean,6.590613e+05,1.838312e+01,2.011955e+03,5.374382e-01,6.396699e-01,5.287778e+00,-8.981353e+00,6.346533e-01,9.281477e-02,3.215370e-01,2.523489e-01,2.230189e-01,4.555636e-01,1.213771e+02,2.495618e+05,3.885879e+00
std,4.285492e+05,1.588554e+01,6.803901e+00,1.844780e-01,2.705009e-01,3.555197e+00,5.682215e+00,4.815275e-01,1.268409e-01,3.549872e-01,3.650731e-01,2.010707e-01,2.685190e-01,2.977975e+01,1.494262e+05,4.676967e-01
min,0.000000e+00,0.000000e+00,2.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00,-5.810000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.073000e+03,0.000000e+00
25%,2.899408e+05,5.000000e+00,2.006000e+03,4.130000e-01,4.540000e-01,2.000000e+00,-1.082900e+01,0.000000e+00,3.710000e-02,6.400000e-03,1.050000e-06,9.790000e-02,2.260000e-01,9.879700e+01,1.810910e+05,4.000000e+00
50%,5.798815e+05,1.500000e+01,2.012000e+03,5.500000e-01,6.940000e-01,5.000000e+00,-7.450000e+00,1.000000e+00,5.070000e-02,1.470000e-01,1.760000e-03,1.340000e-01,4.380000e-01,1.219310e+02,2.257440e+05,4.000000e+00
75%,1.031689e+06,2.900000e+01,2.018000e+03,6.770000e-01,8.730000e-01,8.000000e+00,-5.276000e+00,1.000000e+00,8.900000e-02,6.400000e-01,6.140000e-01,2.920000e-01,6.740000e-01,1.399030e+02,2.869135e+05,4.000000e+00
max,1.473395e+06,1.000000e+02,2.023000e+03,9.930000e-01,1.000000e+00,1.100000e+01,6.172000e+00,1.000000e+00,9.710000e-01,9.960000e-01,1.000000e+00,1.000000e+00,1.000000e+00,2.499930e+02,6.000495e+06,5.000000e+00


In [16]:
data_1m.head()

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4


## Normalizing song + artist columns to create unique common identifier column: song_artist for all dataframes

In [17]:
# normalize column names by changing both dfs to song, artist
data_hits = data_hits.copy()
data_hits.rename(columns={'Song': 'song', 'Artist': 'artist'}, inplace=True)

data_1m = data_1m.copy()
data_1m.rename(columns={'track_name': 'song', 'artist_name': 'artist'}, inplace=True)

df1 = df1.copy()
df1.rename(columns={'name': 'song', 'artists': 'artist'}, inplace=True)

df_hits = df_hits.copy()
df_hits.rename(columns={'Song':'song', 'Performer':'artist'}, inplace=True)

In [18]:
#remove any spaces and normalize potential capitalization discrepancies
data_hits['song'] = data_hits['song'].str.lower().str.strip()
data_hits['artist'] = data_hits['artist'].str.lower().str.strip()

data_1m['song'] = data_1m['song'].str.lower().str.strip()
data_1m['artist'] = data_1m['artist'].str.lower().str.strip()

df1['song'] = df1['song'].str.lower().str.strip()
df1['artist'] = df1['artist'].str.lower().str.strip()

df_hits['song'] = df_hits['song'].str.lower().str.strip()
df_hits['artist'] = df_hits['artist'].str.lower().str.strip()

In [19]:
data_hits['song_artist'] = data_hits['song'] + "_" + data_hits['artist']
data_1m['song_artist'] = data_1m['song'] + "_" + data_1m['artist']
df1['song_artist'] = df1['song'] + "_" + df1['artist']
df_hits['song_artist'] = df_hits['song'] + "_" + df_hits['artist']

### Removing duplicate song/artist pairings

In [20]:
#tracks can be chart toppers many weeks in a row, this removes duplicate songs from the dataset
data_hits_unique = data_hits.drop_duplicates(subset=['song','artist'])
print("total billboard top 100 1958-2024 unique songs:", len(data_hits_unique))

# find if there are any duplicate tracks in this dataset and remove them if they exist
data_1m_unique = data_1m.drop_duplicates(subset=['song','artist'])
print("total 1.2 million track dataset unique songs:", len(data_1m_unique))

# find if there are any duplicate tracks in this dataset and remove them if they exist
df1_unique = df1.drop_duplicates(subset=['song','artist'])
print("total 1 million track dataset unique songs:", len(df1_unique))

#tracks can be chart toppers many weeks in a row, this removes duplicate songs from the dataset
df_hot_unique = df_hits.drop_duplicates(subset=['song','artist'])
print("total hot 100 unique songs:", len(df_hot_unique))

total billboard top 100 1958-2024 unique songs: 31005
total 1.2 million track dataset unique songs: 1151896
total 1 million track dataset unique songs: 1139057
total hot 100 unique songs: 24219


## Combining ~1.2 million song datasets on common features and remove duplicate song_artist values

In [21]:
df1_unique.columns.sort_values()

Index(['acousticness', 'album', 'album_id', 'artist', 'artist_ids',
       'danceability', 'disc_number', 'duration_ms', 'energy', 'explicit',
       'id', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'release_date', 'song', 'song_artist', 'speechiness', 'tempo',
       'time_signature', 'track_number', 'valence', 'year'],
      dtype='object')

In [22]:
data_1m_unique.columns.sort_values()

Index(['Unnamed: 0', 'acousticness', 'artist', 'danceability', 'duration_ms',
       'energy', 'genre', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'song', 'song_artist', 'speechiness', 'tempo',
       'time_signature', 'track_id', 'valence', 'year'],
      dtype='object')

In [23]:
conserved_columns = list(df1_unique.columns.intersection(data_1m_unique.columns))
conserved_columns

['song',
 'artist',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature',
 'year',
 'song_artist']

In [24]:
# combine tracks in 1.2 million song datasets
df_2m = pd.merge(df1_unique[conserved_columns], data_1m_unique[conserved_columns], on=conserved_columns, how='outer')

In [25]:
df_2m.dtypes

song                 object
artist               object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms           int64
time_signature      float64
year                  int64
song_artist          object
dtype: object

In [26]:
len(df_2m)

2271907

In [27]:
# drop duplicate song/artist pairs between the 2 million songs (from combined 1 million song datasets)
df_2m_unique = df_2m.drop_duplicates(subset=['song_artist'], keep='first')
len(df_2m_unique)

2202814

## Pairing song feature data to complete Billboard Top 100 1958-2024 from additional hot 100 dataset for overlapping songs

In [28]:
common_columns = ['song', 'artist', 'song_artist']
top_hits_all = pd.merge(data_hits_unique, df_hot_unique, on=common_columns, how='left')

## Finding overlapping tracks between 2 million songs and 1958-2024 Billboard Top 100

In [29]:
# finding where song/artist pairs from billboard Top 100 overlap with 2.3 million songs
overlap_tracks = df_2m_unique[df_2m_unique['song_artist'].isin(top_hits_all['song_artist'])]
len(overlap_tracks)

11545

In [30]:
overlap_tracks.sort_values(by='song_artist')

,song,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,song_artist
1843264,#1,nelly,0.690,0.592,7,-5.973,1,0.3000,0.02360,0.000000,0.5600,0.466,179.980,198760,4.0,2002,#1_nelly
1256776,#selfie,the chainsmokers,0.789,0.916,0,-3.262,1,0.2490,0.01350,0.000008,0.0770,0.658,127.956,183750,4.0,2014,#selfie_the chainsmokers
654479,'65 love affair,paul davis,0.617,0.691,2,-3.643,0,0.0268,0.35100,0.000001,0.0831,0.915,156.109,219827,4.0,2008,'65 love affair_paul davis
584385,'til i can make it on my own,tammy wynette,0.462,0.300,7,-11.937,1,0.0300,0.77400,0.000004,0.1180,0.151,140.767,181800,4.0,1972,'til i can make it on my own_tammy wynette
2064559,'til my baby comes home,luther vandross,0.813,0.665,11,-7.887,0,0.1630,0.04000,0.000011,0.0276,0.852,139.556,332013,4.0,2007,'til my baby comes home_luther vandross
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592114,yummy,justin bieber,0.676,0.506,9,-6.652,0,0.0958,0.34500,0.000000,0.1180,0.497,145.842,208520,4.0,2020,yummy_justin bieber
1333542,zero,chris brown,0.731,0.818,1,-4.564,0,0.0638,0.05170,0.000000,0.0743,0.812,120.993,214600,4.0,2015,zero_chris brown
222777,zip code,the five americans,0.400,0.446,9,-8.624,1,0.0276,0.02830,0.000269,0.0653,0.963,143.408,152827,4.0,2003,zip code_the five americans
683131,zombie,bad wolves,0.448,0.826,2,-3.244,0,0.0319,0.00756,0.000000,0.1170,0.190,77.093,254805,4.0,2018,zombie_bad wolves


## finalize hit song dataset by pairing song feature data from overlap tracks

In [31]:
# identify columns in common to merge dataframes on
common_columns = list(top_hits_all.columns.intersection(overlap_tracks.columns))
common_columns

['song',
 'artist',
 'year',
 'song_artist',
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature']

In [32]:
# merge top hits dataset + song features with overlapping top tracks from 2.3 mil dataset
# this will maximize the number of hit songs from our source of truth that will have datapoints for song features

top_hits_features = pd.merge(overlap_tracks, top_hits_all, on=common_columns, how='outer')

# drop duplicate columns, duplicate song/artists and songs without feature data available
top_hits_features = top_hits_features.drop_duplicates(subset = 'song_artist', keep='first')
top_hits_features = top_hits_features [[col for col in top_hits_features.columns if not col.endswith('_y')]]

top_hits_features = top_hits_features.dropna()

#### adding binary 0/1 column to overlapping hit songs to identify these as hit songs (1 = hit song, which all of these are)

In [33]:
top_hits_features.loc[:, 'hit_song'] = 1

In [34]:
top_hits_features.head(1)

,song,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,song_artist,hit_song
0,guerrilla radio,rage against the machine,0.599,0.957,11.0,-5.764,1.0,0.188,0.0129,0.000071,0.155,0.489,103.68,206200.0,4.0,1999,guerrilla radio_rage against the machine,1


In [35]:
# export the "unique_overlap_data"
top_hits_features.to_csv('unique_overlapping_hit_songs.csv', index=False)

### Remove overlapping tracks from 2.3 million song dataset

In [36]:
len(df_2m_unique)

2202814

In [37]:
non_hit_tracks = df_2m_unique[~df_2m_unique['song_artist'].isin(top_hits_features['song_artist'])]
len(non_hit_tracks)

2191269

In [38]:
#confirm that the difference between the original dataframe and the filtered one is only the # of overlapping tracks
len(df_2m_unique) - len(non_hit_tracks)

11545

#### adding binary 0/1 column to overlapping hit songs to identify these as non-hit songs (0 = not a hit song)

In [39]:
non_hit_tracks.loc[:, 'hit_song'] = 0

C:\Users\Linds\AppData\Local\Temp\ipykernel_12620\643355793.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_hit_tracks.loc[:, 'hit_song'] = 0


In [40]:
non_hit_tracks.head(1)

,song,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,song_artist,hit_song
0,testify,rage against the machine,0.47,0.978,7,-5.399,1,0.0727,0.0261,0.000011,0.356,0.503,117.906,210133,4.0,1999,testify_rage against the machine,0


In [41]:
# export the non-hit songs dataset of ~2.2 million total tracks
# will be have outliers removed before taking ~24k random tracks for finalized dataset
non_hit_tracks.to_csv('non_hit_songs_2mil.csv', index=False)